In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [3]:
img_width, img_height = 256, 256
train_data_dir        = "data/train"
validation_data_dir   = "data/test"
nb_train_samples      = 2000         # Number of samples to train before going to next epoch
nb_validation_samples = 400          # Number of samples to validate on before next epoch
batch_size            = 32           # Number of samples to run at once, due to memory constraints
epochs                =  5           # Number of cycles of training
n_labels              = 19           # Number of labels to fit
dense_final_layer     = 512          # Final layer number of nodes
final_dropout         = 0.5          # Dropout fraction
learning_rate         = 1e-4
momentum              = 0.9

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

NameError: name 'applications' is not defined

In [3]:
for layer in model.layers:
    print layer

In [6]:
"""
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)       295168    
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 64, 64, 256)       590080    
_________________________________________________________________
block3_conv3 (Conv2D)        (None, 64, 64, 256)       590080    
_________________________________________________________________
block3_conv4 (Conv2D)        (None, 64, 64, 256)       590080    
_________________________________________________________________
block3_pool (MaxPooling2D)   (None, 32, 32, 256)       0         
_________________________________________________________________
block4_conv1 (Conv2D)        (None, 32, 32, 512)       1180160   
_________________________________________________________________
block4_conv2 (Conv2D)        (None, 32, 32, 512)       2359808   
_________________________________________________________________
block4_conv3 (Conv2D)        (None, 32, 32, 512)       2359808   
_________________________________________________________________
block4_conv4 (Conv2D)        (None, 32, 32, 512)       2359808   
_________________________________________________________________
block4_pool (MaxPooling2D)   (None, 16, 16, 512)       0         
_________________________________________________________________
block5_conv1 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_conv2 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_conv3 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_conv4 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_pool (MaxPooling2D)   (None, 8, 8, 512)         0         
=================================================================
Total params: 20,024,384.0
Trainable params: 20,024,384.0
Non-trainable params: 0.0
"""

'\nLayer (type)                 Output Shape              Param #   \n=================================================================\ninput_1 (InputLayer)         (None, 256, 256, 3)       0         \n_________________________________________________________________\nblock1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      \n_________________________________________________________________\nblock1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     \n_________________________________________________________________\nblock1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         \n_________________________________________________________________\nblock2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     \n_________________________________________________________________\nblock2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    \n_________________________________________________________________\nblock2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [4]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers:
    layer.trainable = False

#Adding custom Layers, FUCK WITH THIS
x = model.output
x = Flatten()(x)
x = Dense(dense_final_layer, activation="relu")(x)
x = Dropout(final_dropout)(x)
x = Dense(dense_final_layer, activation="relu")(x)
predictions = Dense(n_labels, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(
                        loss      = "categorical_crossentropy", 
                        optimizer = optimizers.SGD(
                                                    lr=learning_rate, 
                                                    momentum=momentum
                                                  ), 
                        metrics=["accuracy"])

/home/sean/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [5]:
# Initiate the train and test generators with data Augumentation 
# DIRECTORY IS THE LABEL!!!!!!!!!!!!!!!!!
train_datagen = ImageDataGenerator(
                                    rescale = 1./255,
                                    horizontal_flip = True,
                                    fill_mode = "nearest",
                                    zoom_range = 0.3,
                                    width_shift_range = 0.3,
                                    height_shift_range=0.3,
                                    rotation_range=30,
                                  )

test_datagen = ImageDataGenerator(
                                    rescale = 1./255,
                                    horizontal_flip = True,
                                    fill_mode = "nearest",
                                    zoom_range = 0.3,
                                    width_shift_range = 0.3,
                                    height_shift_range=0.3,
                                    rotation_range=30
                                 )




train_generator = train_datagen.flow_from_directory(
                                                        train_data_dir,
                                                        target_size = (img_height, img_width),
                                                        batch_size = batch_size, 
                                                        class_mode = "categorical"
                                                   )



validation_generator = test_datagen.flow_from_directory(
                                                        validation_data_dir,
                                                        target_size = (img_height, img_width),
                                                        class_mode = "categorical"
                                                       )

Found 4318 images belonging to 19 classes.
Found 1021 images belonging to 19 classes.


In [6]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint(
                                "vgg16_1.h5", 
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='auto', 
                                period=1
                            )

early = EarlyStopping(
                        monitor='val_acc', 
                        min_delta=0, 
                        patience=10, 
                        verbose=1, 
                        mode='auto'
                     )

In [ ]:
# Train the model 
model_final.fit_generator(
                            train_generator,
                            samples_per_epoch = nb_train_samples,
                            epochs = epochs,
                            validation_data = validation_generator,
                            nb_val_samples = nb_validation_samples,
                            callbacks = [checkpoint, early]
                         )

/home/sean/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/sean/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=257, epochs=50, callbacks=[<keras.ca..., validation_steps=466)`


Epoch 1/50
  3/257 [..............................] - ETA: 37:11 - loss: 3.0640 - acc: 0.1458

/home/sean/anaconda/lib/python2.7/site-packages/PIL/Image.py:918: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


 11/257 [>.............................] - ETA: 37:55 - loss: 3.0230 - acc: 0.1364